### PDF文档解析

In [44]:
import re
import pandas as pd
from typing import List, Union
from pathlib import Path
from tqdm import tqdm
from src.modules.document.pdf_parser import PdfParserUsingPyMuPDF


def cleanup(document: List[str]):
    """
    清洗数据 
    """
    # '\n', ' ' 删除
    text = text.replace('\n','').replace(' ','')
        
    
    # 删除本文档为2024CCFBDC***
    head_pattern = '本文档为2024CCFBDCI比赛用语料的一部分。[^\s]+仅允许在本次比赛中使用。'
    
    # news_pattern
    pattern1 = r"发布时间：[^\s]+发布人：新闻宣传中心"
    pattern2 = r"发布时间：[^\s]+发布人：新闻发布人"
    pattern3 =  r'发布时间：\d{4}年\d{1,2}月\d{1,2}日'
    news_pattern = head_pattern+'|'+pattern1+'|'+pattern2+'|'+pattern3
    text = re.sub(news_pattern,'',text)

    # report_pattern
    report_pattern1 = '第一节重要提示[^\s]+本次利润分配方案尚需提交本公司股东大会审议。'
    report_pattern12 = '一重要提示[^\s]+股东大会审议。'
    report_pattern13 = '一、重要提示[^\s]+季度报告未经审计。'
    report_pattern2 = '本公司董事会及全体董事保证本公告内容不存在任何虚假记载、[^\s]+季度财务报表是否经审计□是√否'
    report_pattern3 = '中国联合网络通信股份有限公司（简称“公司”）董事会审计委员会根据相关法律法规、[^\s]+汇报如下：'
    report_pattern = report_pattern1+'|'+report_pattern12+'|'+report_pattern13+'|'+report_pattern2+'|'+report_pattern3
    text = re.sub( report_pattern,'',text)
#     white paper 版本一 效果不好
    # 优先级别 bp1 bp2 bp3
#     bp_pattern_law = '版权声明[^\s]+追究其相关法律责任。'
#     bp_pattern1 = r'目录.*?披露发展报告（\d{4}年）' # 只针对AZ08.pdf
#     bp_pattern2 = r'目录.*?白皮书.*?（\d{4}年）'
#     bp_pattern3 = r'目录.*?白皮书'
#     bp_pattern = bp_pattern_law  +'|'+bp_pattern1+'|'+bp_pattern2+'|'+bp_pattern3
#     text = re.sub(bp_pattern,'',text)
#     print(text)
    
    return text


__news__ = ["AF", "AT", "AW"]
__report__ = ["AZ", "AY"]

data = []
pdf_parser = PdfParserUsingPyMuPDF()
files: List[Path] = list(Path("dataset/A榜/A_document/").rglob("*.pdf"))
for i, pdf_file in enumerate(files):
    code = pdf_file.name[:2]
    if code in __news__:
        doc = pdf_parser.parse(pdf_file)
        cleaned = cleanup_news(pdf_file.name, doc)
        data.append(cleaned)
        print(pdf_file, cleaned.get("发布时间"), cleaned.get("标题"), len(cleaned.get("内容")))

df = pd.DataFrame(data)
print(df.head())
df.to_csv("文档结构化.csv", index=False)

dataset/A榜/A_document/AT08.pdf 2021-03-05 使命担当5G赋能中国联通实力护航“智慧冬奥” 1997
dataset/A榜/A_document/AT34.pdf 2023-09-26 中国联通：推深抓实“五个贯穿”奋力书写主题教育联通精彩答卷 4849
dataset/A榜/A_document/AT20.pdf 2017-12-26 中国联通成为北京2022年冬奥会和冬残奥会官方通信服务合作伙伴 1037
dataset/A榜/A_document/AT21.pdf 2023-06-14 深化主题教育践行宗旨为民造福丨中国联通：多措并举筑牢反诈防骗坚固防线 1617
dataset/A榜/A_document/AT35.pdf 2023-10-13 中国联通：为全球数字化转型注入新动能 2563
dataset/A榜/A_document/AT09.pdf 2024-07-20 算网筑基拥抱智能 2441
dataset/A榜/A_document/AT23.pdf 2024-02-06 学深悟透强根基实干笃行谱新篇——中国联通扎实开展主题教育推动高质量发展综述 7722
dataset/A榜/A_document/AT22.pdf 2023-06-29 深化主题教育数字化赋能乡村振兴建新功丨千年古村复繁华数字赋能新慈乌 2636
dataset/A榜/A_document/AT26.pdf 2019-04-26 以高质量发展铸造更畅通的“信息丝绸之路”中国联通四大举措服务“一带一路”建设 2607
dataset/A榜/A_document/AT32.pdf 2024-01-23 中国联通：坚持问题导向推动调研成果转化为高质量发展动力 3545
dataset/A榜/A_document/AT33.pdf 2023-06-01 中国联通：同题共答同向发力以严督实导推动主题教育扎实开展 1439
dataset/A榜/A_document/AT27.pdf 2017-07-24 用得起、用得好、用得放心中国联通发力提升国际漫游用户感知 2937
dataset/A榜/A_document/AT31.pdf 2023-05-22 中国联通：坚持“学在深处谋在新处干在实处”推动主题教育落地落实 1854
dataset/A榜/A

In [43]:
def cleanup_news(fnm: str, document: Union[str, List[str]]) -> dict:
    """
    清洗新闻
    """
    if isinstance(document, list):
        document = "".join(document)
    # 使用正则表达式提取标题、发布时间、发布人和内容
    title_match = re.search(r'本文档.*?仅允许在本次比赛中使用。\n(.*?)\n', document, re.MULTILINE | re.DOTALL)
    pub_time_match = re.search(r'发布时间：(\d{4}-\d{2}-\d{2}|\d{4} 年\d{1,2} 月\d{1,2} 日)', document, re.MULTILINE | re.DOTALL)
    content_match = re.search(r'(\d{4}-\d{2}-\d{2}|\d{4} 年\d{1,2} 月\d{1,2} 日)(.*)', document, re.MULTILINE | re.DOTALL)
    author_match = re.search(r'(发布人|文章来源)：(.*?)(?=\n)', document, re.MULTILINE | re.DOTALL)
    author_match_delete = re.search(r'((发布人|文章来源)：.*?(?=\n))', document, re.MULTILINE | re.DOTALL)
    
    # 提取数据
    title = title_match.group(1).strip() if title_match else ""
    pub_time = pub_time_match.group(1).strip() if pub_time_match else ""
    content = content_match.group(2).strip() if content_match else ""
    author = author_match.group(2).strip() if author_match else ""
    author_match = author_match_delete.group(1).strip() if author_match else ""
    
    # 结果字典
    return {
        "文件名": fnm,
        "标题": title.replace("\n", "").replace(" ", ""),
        "内容": content.replace("\n", "").replace(" ", "").replace(author_match, ""),
        "发布时间": pub_time.replace("\n", "").replace(" ", ""),
        "发布人": author.replace("\n", "").replace(" ", "")
    }

# f = Path("dataset/A榜/A_document/AW09.pdf")
f = Path("dataset/A榜/A_document/AT06.pdf")
TEMP_NEWS_TEXT = "".join(PdfParserUsingPyMuPDF().parse(f))
print([TEMP_NEWS_TEXT])
cleanup_news(f.name, TEMP_NEWS_TEXT)


['本文档为2024 CCF BDCI 比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况\n不符，可能不具备现实意义，仅允许在本次比赛中使用。\n山西联通：深化落实“六个一”机制助推企业高质量发展\n发布时间：2022-01-14 发布人：新闻宣传中心\n习近平总书记指出：党支部是团结群众的核心、教育党员的学校、攻坚克难的堡垒。朔州联通坚持以党建为统领，深入贯\n彻落实党的十九届六中全会精神，深化落实“六个一”工作机制，让党支部在基层党建工作中唱主角，在不断融入中心工作中提高\n党建质量，在持续推动企业高质量发展中彰显党建力量。\n党建引领，同向发力，画好“同心圆\n朔州联通党委亲自部署，相关部门统筹协调，上下协同、同向发力，以党建为引领，以“六个一”机制为抓手，不断把党建\n与中心工作融合推向深入，画好高质量发展同心圆。\n一是“选好头雁”强基础。按照“四同步”“四对接”要求，选优配好党支部书记，配齐配强支委会委员，切实做到20 个支部行\n政领导班子与党支部领导班子相匹配，充分发挥“领头雁”示范引领作用，党支部工作与中心工作同研究、同部署、同推动、同检\n查、同考核，为确保党支部能够担事、支部能够议事、党员能够干事打好基础。\n二是“强基固本”植沃土。围绕构建高质量发展新格局，以《山西联通基层党支部“六化”建设标准》为标尺，通过“常态化自\n查+不定期督导”方式，促进基层党支部对照不足扎实整改，不断提升党支部工作规范化、科学化水平。同时以争创“五星党支部”\n为牵引，以“六化”达标为前提，党建推动中心工作成效为检验，全面铸就过硬战斗堡垒，为“六个一”机制出成效厚植沃土，以高\n质量党建引领高质量发展。\n三是“考核评价”促提升。支部层面以“党建+”模式贯穿全年工作，要求各党支部基础党建必须与中心工作相结合。设置KPI\n考核分值及否决制推动党建与中心工作结合，通过“党建+绩效”的延伸考核，促进党支部抓党建、促提升、谋发展的能力。党员\n层面以“民主评议”为抓手，把党员个人评价结果纳入评优评先、人才选拔应用，营造担当作为、干事创业、创先争优的良好氛围。\n四是“支撑赋能”有保障。公司领导班子躬身入局、包片帮扶，与“我为群众办实事”结合起来，切实解决急难愁盼问题28\n件，同时成立了三个帮扶工作小组，党务、专业骨干人员深入各基层党支部，从支部的

{'文件名': 'AT06.pdf',
 '标题': '山西联通：深化落实“六个一”机制助推企业高质量发展',
 '内容': '习近平总书记指出：党支部是团结群众的核心、教育党员的学校、攻坚克难的堡垒。朔州联通坚持以党建为统领，深入贯彻落实党的十九届六中全会精神，深化落实“六个一”工作机制，让党支部在基层党建工作中唱主角，在不断融入中心工作中提高党建质量，在持续推动企业高质量发展中彰显党建力量。党建引领，同向发力，画好“同心圆朔州联通党委亲自部署，相关部门统筹协调，上下协同、同向发力，以党建为引领，以“六个一”机制为抓手，不断把党建与中心工作融合推向深入，画好高质量发展同心圆。一是“选好头雁”强基础。按照“四同步”“四对接”要求，选优配好党支部书记，配齐配强支委会委员，切实做到20个支部行政领导班子与党支部领导班子相匹配，充分发挥“领头雁”示范引领作用，党支部工作与中心工作同研究、同部署、同推动、同检查、同考核，为确保党支部能够担事、支部能够议事、党员能够干事打好基础。二是“强基固本”植沃土。围绕构建高质量发展新格局，以《山西联通基层党支部“六化”建设标准》为标尺，通过“常态化自查+不定期督导”方式，促进基层党支部对照不足扎实整改，不断提升党支部工作规范化、科学化水平。同时以争创“五星党支部”为牵引，以“六化”达标为前提，党建推动中心工作成效为检验，全面铸就过硬战斗堡垒，为“六个一”机制出成效厚植沃土，以高质量党建引领高质量发展。三是“考核评价”促提升。支部层面以“党建+”模式贯穿全年工作，要求各党支部基础党建必须与中心工作相结合。设置KPI考核分值及否决制推动党建与中心工作结合，通过“党建+绩效”的延伸考核，促进党支部抓党建、促提升、谋发展的能力。党员层面以“民主评议”为抓手，把党员个人评价结果纳入评优评先、人才选拔应用，营造担当作为、干事创业、创先争优的良好氛围。四是“支撑赋能”有保障。公司领导班子躬身入局、包片帮扶，与“我为群众办实事”结合起来，切实解决急难愁盼问题28件，同时成立了三个帮扶工作小组，党务、专业骨干人员深入各基层党支部，从支部的薄弱点切入，基础点做起，推动各种资源力量落实到支部，助力“六个一”工作机制落实落细，努力把每一个基层党支部建设成为坚强的战斗堡垒。强化教育，凝心聚力，提振“精气神”加强党支部建设，思想建设是重中之重。2021年以热烈庆祝